<a href="https://colab.research.google.com/github/karino2/US-patent-analysis/blob/bert_interactive/colab/bert_interactive_benchmark_2000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Interacitve evaluation notebook for BERT with grants2000**

Input app claim (from 2017, cited from 2012) and show most similar claim.

In [0]:
import os
import datetime
import pickle
import gzip
import pandas as pd
import numpy as np

import tensorflow as tf

In [2]:
tf.__version__

'1.12.0'

In [15]:
!git clone -b docker https://github.com/yoheikikuta/bert.git

Cloning into 'temp'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 174.52 KiB | 2.60 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [4]:
!ls

adc.json  bert	sample_data


# Data setup

Data setup is based on https://github.com/karino2/US-patent-analysis/blob/bert_input/colab/bert_tokenized_claim_setup.ipynb

We only use grants tokens. (though file both contains applications and grants)

In [0]:
!mkdir ./bert/data

In [0]:
from google.colab import auth
auth.authenticate_user()

In [6]:
!gsutil cp gs://karino2-uspatent/features/test_grants_ids.pkl.gz ./bert/data/

Copying gs://karino2-uspatent/features/test_grants_ids.pkl.gz...
/ [1 files][  3.0 MiB/  3.0 MiB]                                                
Operation completed over 1 objects/3.0 MiB.                                      


In [0]:
!ls ./bert/data

In [0]:
with gzip.open("./bert/data/test_grants_ids.pkl.gz", 'rb') as f:
     (_, grants_ids) = pickle.load(f)

In [0]:
!gsutil cp gs://karino2-uspatent/citations_info_2000.df.gz ./bert/data/
!gsutil cp gs://karino2-uspatent/testset_app_1000.df.gz ./bert/data/
!gsutil cp gs://karino2-uspatent/training_app_1000.df.gz ./bert/data/
!gsutil cp gs://karino2-uspatent/grants_for_2000.df.gz ./bert/data/

In [0]:
citations_info_target = pd.read_pickle("./bert/data/citations_info_2000.df.gz")
# training_app_df = pd.read_pickle("./bert/data/training_app_1000.df.gz")
testset_app_df = pd.read_pickle("./bert/data/testset_app_1000.df.gz")
grants_target_df = pd.read_pickle("./bert/data/grants_for_2000.df.gz")

**Check tpu name**

In [9]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.29.55.130:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 15700132874213056335),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2438556059540875912),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 7982358489383469006),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7316591919784883874),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4845482972589313628),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 6332370725057830377),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 2425035318090804676),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 17626031719690383485),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 160672

### Bert setup

In [0]:
VOCAB_CONFIG_PATH=' gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12' #@param {type:"string"}

In [11]:
!gsutil cp {VOCAB_CONFIG_PATH}/vocab.txt ./bert/model/patent/vocab.txt
!gsutil cp {VOCAB_CONFIG_PATH}/bert_config.json ./bert/model/patent/bert_config.json

Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/vocab.txt...
- [1 files][226.1 KiB/226.1 KiB]                                                
Operation completed over 1 objects/226.1 KiB.                                    
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_config.json...
/ [1 files][  313.0 B/  313.0 B]                                                
Operation completed over 1 objects/313.0 B.                                      


In [12]:
!ls ./bert/model/patent

bert_config.json  vocab.txt


In [0]:
import sys
sys.path.append("./bert")

In [0]:
import tokenization

In [0]:
tokenizer = tokenization.FullTokenizer(
    vocab_file="./bert/model/patent/vocab.txt", do_lower_case=True)

In [0]:
CLS_ID = tokenizer.vocab["[CLS]"]
SEP_ID = tokenizer.vocab["[SEP]"]

In [0]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

In [0]:
label_list = ["not_entailment", "entailment"]
max_seq_length = 512

label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i

In [0]:
def original_convert_single_pair(ids_a_input, ids_b_input):

  # Clone and pop for truncate. Most of the case result len is the same for our purpose, but try simple implementation first.
  ids_a = list(ids_a_input)
  ids_b = list(ids_b_input)
    
  # Modifies `tokens_a` and `tokens_b` in place so that the total
  # length is less than the specified length.
  # Account for [CLS], [SEP], [SEP] with "- 3"
  _truncate_seq_pair(ids_a, ids_b, max_seq_length - 3)

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  input_ids = []
  segment_ids = []
  input_ids.append(CLS_ID)
  segment_ids.append(0)
  for token in ids_a:
    input_ids.append(token)
    segment_ids.append(0)
  input_ids.append(SEP_ID)
  segment_ids.append(0)

  for token in ids_b:
    input_ids.append(token)
    segment_ids.append(1)
  input_ids.append(SEP_ID)
  segment_ids.append(1)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  return (input_ids, input_mask, segment_ids)


### Optimization for our application

dup ids consume huge memory.
We cach and use the same memory if ids_a_nput and ids_b_input have enough size.

In [0]:
NORMAL_INPUT_MASK = [1]*max_seq_length
NORMAL_SEGMENT_IDS = [0]*257+[1]*255

In [0]:
# [CLS](0), 1-255(len=255), [SEP], 257-510(len=254), [SEP]

In [0]:
LEN_255_CACHE = {}
LEN_254_CACHE = {}

In [0]:
def truncage_with_cache(ids, target_len):
  if target_len == 255:
    cache = LEN_255_CACHE
  else:
    assert target_len == 254
    cache = LEN_254_CACHE
  if ids not in cache:
    cache[ids] = ids[0:target_len]
  return cache[ids]

In [0]:
def fast_convert_single_pair(ids_a_input, ids_b_input):  
  if (len(ids_a_input) < 256) or (len(ids_b_input) < 256) :
    return original_convert_single_pair(ids_a_input, ids_b_input)
  
  ids_a = truncage_with_cache(ids_a_input, 255)
  ids_b = truncage_with_cache(ids_b_input, 254)

  input_ids = (CLS_ID,) + ids_a + (SEP_ID,) + ids_b + (SEP_ID,)
  
  return (input_ids, NORMAL_INPUT_MASK, NORMAL_SEGMENT_IDS)


In [0]:
def feature_tuplist_to_feature_dict(ftups):
    dic = {}
    dic['input_ids'] = [tup[0] for tup in ftups]
    dic["input_mask"] = [tup[1] for tup in ftups]
    dic["segment_ids"] = [tup[2] for tup in ftups]
    dic["label_ids"] = [[0] for _ in ftups]
    return dic

In [0]:
# convert to tuple for optimization.
grants_ids_tup = [tuple(ids) for ids in grants_ids]

Tokenize claim

In [0]:
import re

CLAIM_PAT = re.compile(r'<claims[^>]*>(.*)</claims>',re.MULTILINE|re.DOTALL)
TAG_PAT = re.compile(r"<.*?>")


def whole_xml_to_claim_xml(whole):
    mat = CLAIM_PAT.search(whole)
    return mat.group(1)
  
  
def whole_xml_to_claim(whole):
    return TAG_PAT.sub(' ', whole_xml_to_claim_xml(whole))

In [0]:
def claim_to_ids(claim):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(claim))

# Model setup

In [0]:
import os

import modeling
import tokenization

from run_classifier import model_fn_builder


In [0]:
bert_config = modeling.BertConfig.from_json_file("./bert/model/patent/bert_config.json")

In [123]:
BUCKET = 'karino2-uspatent' #@param {type:"string"}
OUTPUT_DIR = 'gs://{}/mlstudy-phys/bert/models/patent_1000'.format(BUCKET)

INIT_CHECK_POINT_FILE='model.ckpt-1602' #@param {type:"string"}
INIT_CHECK_POINT = '{}/{}'.format(OUTPUT_DIR, INIT_CHECK_POINT_FILE)

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))
print('***** Init checkpoint: {} *****'.format(INIT_CHECK_POINT))

***** Model output directory: gs://karino2-uspatent/mlstudy-phys/bert/models/patent_1000 *****
***** Init checkpoint: gs://karino2-uspatent/mlstudy-phys/bert/models/patent_1000/model.ckpt-1602 *****


In [0]:
class FLAGS(object):
    '''Parameters.'''
    def __init__(self):
        self.vocab_file = "./bert/model/patent/vocab.txt"
        self.use_tpu = True
        self.output_dir = OUTPUT_DIR
        # self.data_dir = "./bert/data/1000"
        self.init_checkpoint = INIT_CHECK_POINT
        self.predict_batch_size = 16
        # TPU related
        self.num_tpu_cores = 8
        self.tpu_name = TPU_ADDRESS
        
        # following parameters are not used anymore. (because we create feature by hand)
        self.do_lower_case = True
        self.max_seq_length = 512
        
        # The following parameters are not used in predictions.
        # Just use to create RunConfig.
        self.master = None
        self.save_checkpoints_steps = 1
        self.learning_rate = 0
        self.num_warmup_steps = 0
        self.num_train_steps = 0
        # TPU related. Some of these value have positive int not to make TPUEstimator angry (even though these are not used...).
        self.eval_batch_size = 32
        self.train_batch_size = 32 
        self.iterations_per_loop = 1000


In [0]:
FLAGS = FLAGS()

In [0]:
# tokenizer = tokenization.FullTokenizer(
#    vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

# tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=FLAGS.tpu_name).get_master()
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=FLAGS.tpu_name)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

In [0]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(["not_entailment", "entailment"]),
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=FLAGS.num_train_steps,
    num_warmup_steps=FLAGS.num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_tpu)

In [0]:
# https://github.com/kyzhouhzau/BERT-NER/issues/19

def model_fn_wrapper(features, labels, mode, params):
  tspec = model_fn(features, labels, mode, params)
  pred_dict = {'predictions': tspec.predictions}
  return tf.contrib.tpu.TPUEstimatorSpec(
          mode=tspec.mode, predictions=pred_dict, scaffold_fn=tspec.scaffold_fn)


In [129]:
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    # model_fn=model_fn,
    model_fn=model_fn_wrapper,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

INFO:tensorflow:Using config: {'_model_dir': 'gs://karino2-uspatent/mlstudy-phys/bert/models/patent_1000', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.29.55.130:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f92b53e5da0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'grpc://10.29.55.130:8470', '_evaluation_master': b'grpc://10.29.55.130:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, nu

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
def predict_fdicts(fdict):
  def _batch_input_function(params):
    test_ds = tf.data.Dataset.from_tensor_slices(fdict)    
    return test_ds.batch(params['batch_size'])
  
  result = estimator.predict(_batch_input_function)
  print(datetime.datetime.today())
  result = list(result)
  print(datetime.datetime.today())
  
  return  [pred['predictions'] for pred in result]


# Interactive view

In [0]:
def results_to_result_df(results):
  results_arr = np.array(results)
  results_df = grants_target_df.copy()
  results_df["score"] = results_arr[:, 1]
  return results_df.sort_values("score", ascending=False)

In [0]:
from google.colab import widgets
from google.colab import output


In [0]:
def show_candidates(result_df, app_id):
  print('Show 10 most probable candidate for app({})'.format(app_id))

  candidate_df = result_df.sort_values("score", ascending=False)[0:10]


  tab_labels = list(candidate_df.apply(lambda row: "{}:{:.6f}".format(row.parsed, row.score), axis=1))

  tb = widgets.TabBar(tab_labels, location="top")
  for i, _ in enumerate(tab_labels):
    with tb.output_to(i):
      print(whole_xml_to_claim(candidate_df["xml"].values[i]))

### How to use

1. Eval cell below
2. Choose which application you want to search cited candidate
3. Choose dropdown of index which you want to search (left most tab is 0)
4. Eval cell below of dropdown. This cell takes about 40 seconds.
5. Show the results.

In [99]:
print('Random 10 sampling of application claims. (re-sampling each time cell eval)')


sampled_apps = testset_app_df.sample(10)


app_ids = list(map(lambda s: str(s), sampled_apps.app_id.values))

tb = widgets.TabBar(app_ids, location="top")
for i, _ in enumerate(app_ids):
  with tb.output_to(i):
    print(whole_xml_to_claim(sampled_apps["xml"].values[i]))
    


Random 10 sampling of application claims. (re-sampling each time cell eval)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


 
  1 . A retention member comprising:
 a plurality of inwardly curved blocking tab sections connected to each other by outwardly curved connecting sections, and 
 arm sections at each end of the retention member; 
 wherein the arm sections are spaced apart from each other. 
 
 
 
  2 . The retention member of  claim 1  wherein said bone anchor member comprises a head portion and a threaded shank portion, and one or more of the inwardly curved blocking tab sections are configured and dimensioned to cover a portion of the top surface of a bone anchor member. 
 
 
  3 . The retention member of  claim 2  wherein first and second inwardly curved blocking tab sections cover portions of the bone anchor member and prevent longitudinal movement of the bone anchor member. 
 
 
  4 . The retention member of  claim 3  wherein the first and second inwardly curved blocking tab sections cover portions of the bone anchor member and resist screw removal when a range of 4-7 inch pounds removal force i

<IPython.core.display.Javascript object>

In [183]:
#@title 検索する出願のインデックス（上記タブの0から9番目のどれか）を選ぶ { run: "auto", vertical-output: true }

target_claim_index = "7" #@param [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Selected result becomes string...
target_claim_idx = int(target_claim_index)
print("target app: {}".format(app_ids[target_claim_idx]))
target_claim = whole_xml_to_claim(sampled_apps["xml"].values[target_claim_idx])
print("claim: {}...".format(target_claim[0:100]))

target app: 12564725
claim: 
 
  1 . A terminal device including a display section and an operating section, which is capable of...


In [191]:
target_claim_ids = tuple(claim_to_ids(target_claim))
pairs = [(target_claim_ids, grants_claim) for grants_claim in grants_ids_tup]
fdicts = feature_tuplist_to_feature_dict([fast_convert_single_pair(tup[0], tup[1]) for tup in pairs])

# The below line takes about 40 seconds
results = predict_fdicts(fdicts)

results_df = results_to_result_df(results)
show_candidates(results_df, app_ids[target_claim_idx])

2019-01-10 07:55:24.358031
2019-01-10 07:55:57.876000
Show 10 most probable candidate for app(12564725)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


 
 1. A video processing apparatus comprising:
 first storage control means for, when a video signal including a first image and a second image temporally inserted in the first image is given, controlling storing of the first image of the video signal, on the basis of the temporal position of at least one of the first image and the second image in the video signal; 
 second storage control means for controlling of storing the second image on the basis of the temporal position of at least one of the first image and the second image in the video signal; 
 image detection means for detecting the second image from the video signal, 
 wherein the image detection means include:
 a transition detecting unit that detects, from the video signal, a plurality of image transition frames for determining candidates of the second image signal, and 
 a judgment unit that selects a candidate with a minimum broadcast period as the second image for a temporal position when more than one candidate starts

<IPython.core.display.Javascript object>